# <center> Домашнее задание №8. Решение

## <center> Реализация онлайн-регрессора

Здесь мы реализуем регрессор, обучаемый стохастическим градиентным спуском (SGD). Заполните пропущенный код. Если всё сделано правильно, вы пройдёте простой встроенный тест.

## <center>Линейная регрессия и стохастический градиентный спуск

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.base import BaseEstimator
from sklearn.metrics import mean_squared_error, log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

Реализуйте класс `SGDRegressor`. Спецификация:
- класс наследуется от `sklearn.base.BaseEstimator`
- конструктор принимает параметры `eta` — шаг градиентного спуска ($10^{-3}$ по умолчанию) и `n_epochs` — число проходов по датасету (3 по умолчанию)
- конструктор также создаёт списки `mse_` и `weights_` для отслеживания среднеквадратичной ошибки и вектора весов в процессе итераций градиентного спуска
- класс имеет методы `fit` и `predict`
- метод `fit` принимает матрицу `X` и вектор `y` (объекты `numpy.array`), добавляет столбец единиц к `X` слева, инициализирует вектор весов `w` **нулями** и затем выполняет `n_epochs` итераций обновления весов, на каждой итерации записывая среднеквадратичную ошибку и вектор весов `w` в соответствующие списки, созданные в конструкторе.
- дополнительно метод `fit` создаёт переменную `w_` для хранения весов, при которых достигается минимальная среднеквадратичная ошибка
- метод `fit` возвращает текущий экземпляр класса `SGDRegressor`, то есть `self`
- метод `predict` принимает матрицу `X`, добавляет столбец единиц слева и возвращает вектор предсказаний, используя вектор весов `w_`, созданный методом `fit`.

In [ ]:
class SGDRegressor(BaseEstimator):
    
    def __init__(self, eta=1e-3, n_epochs=3):
        self.eta = eta
        self.n_epochs = n_epochs
        self.mse_ = []
        self.weights_ = []
        
    def fit(self, X, y):
        # добавляем столбец единиц слева от X
        X = np.hstack([np.ones([X.shape[0], 1]), X])
        
        # инициализируем w нулями, размерность (d + 1)
        w = np.zeros(X.shape[1])
        
        for it in tqdm(range(self.n_epochs)):
            for i in range(X.shape[0]):
                
                # new_w используется для одновременного обновления w_0, w_1, ..., w_d
                new_w = w.copy()
                # отдельная (более простая) формула для w_0
                new_w[0] += self.eta * (y[i] - w.dot(X[i, :]))
                for j in range(1, X.shape[1]):
                    new_w[j] += self.eta * (y[i] - w.dot(X[i, :])) * X[i, j]  
                w = new_w.copy()
                
                # сохраняем текущий вектор весов
                self.weights_.append(w)
                # сохраняем текущее значение функции потерь
                self.mse_.append(mean_squared_error(y, X.dot(w)))
        # "лучший" вектор весов
        self.w_ = self.weights_[np.argmin(self.mse_)]
                
        return self
                  
    def predict(self, X):
        # добавляем столбец единиц слева от X
        X = np.hstack([np.ones([X.shape[0], 1]), X])
        # линейное предсказание
        return X.dot(self.w_)

Протестируем алгоритм на данных о росте и весе. Будем предсказывать рост (в дюймах) по весу (в фунтах).

In [ ]:
data_demo = pd.read_csv('../data/weights_heights.csv')

In [ ]:
plt.scatter(data_demo['Weight'], data_demo['Height']);
plt.xlabel('Weight (lbs)')
plt.ylabel('Height (Inch)')
plt.grid();

In [ ]:
X, y = data_demo['Weight'].values, data_demo['Height'].values

Выполните разбиение на train/test и масштабируйте данные.

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                                     test_size=0.3,
                                                     random_state=17)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape([-1, 1]))
X_valid_scaled = scaler.transform(X_valid.reshape([-1, 1]))

Обучите созданный `SGDRegressor` на данных `(X_train_scaled, y_train)`. Оставьте значения параметров по умолчанию.

In [ ]:
sgd_reg = SGDRegressor()
sgd_reg.fit(X_train_scaled, y_train)

Постройте график процесса обучения — зависимость среднеквадратичной ошибки от номера итерации SGD.

In [ ]:
plt.plot(range(len(sgd_reg.mse_)), sgd_reg.mse_)
plt.xlabel('# обновлений')
plt.ylabel('MSE');

Выведите минимальное значение среднеквадратичной ошибки и лучший вектор весов.

In [ ]:
np.min(sgd_reg.mse_), sgd_reg.w_

Постройте график изменения весов модели ($w_0$ и $w_1$) в процессе обучения.

In [ ]:
plt.subplot(121)
plt.plot(range(len(sgd_reg.weights_)), 
         [w[0] for w in sgd_reg.weights_]);
plt.subplot(122)
plt.plot(range(len(sgd_reg.weights_)), 
         [w[1] for w in sgd_reg.weights_]);

Сделайте предсказание на отложенной выборке `(X_valid_scaled, y_valid)` и проверьте значение MSE.

In [ ]:
sgd_holdout_mse = mean_squared_error(y_valid, 
                                        sgd_reg.predict(X_valid_scaled))
sgd_holdout_mse

Сделайте то же самое для класса `LinearRegression` из `sklearn.linear_model`. Оцените MSE на отложенной выборке.

In [ ]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression().fit(X_train_scaled, y_train)
print(lm.coef_, lm.intercept_)
linreg_holdout_mse = mean_squared_error(y_valid, 
                                        lm.predict(X_valid_scaled))
linreg_holdout_mse

In [ ]:
try:
    assert (sgd_holdout_mse - linreg_holdout_mse) < 1e-4
    print('Правильно!')
except AssertionError:
    print("Что-то не так.\n MSE LinearRegression на отложенной выборке: {}"
          "\n MSE SGD на отложенной выборке: {}".format(linreg_holdout_mse, 
                                            sgd_holdout_mse))